# Installations and Imports

In [1]:
!pip install pyvista panel -q
!pip install -q piglet pyvirtualdisplay

In [2]:
import dolfinx
import dolfinx.plot
import numpy as np
import ufl
from petsc4py import PETSc
from mpi4py import MPI
from math import pi, sin, cos, ceil
import pyvista

# Function Definitions

## Main Subroutine

In [3]:
# Function which performs loading for a particular set of parameters:
def perform_loading(L, W, elem_size, E, lambda_, y_rot, x_rot, nu, rho, g, elem_order):
    
    NL, NW = ceil(L/elem_size), ceil(W/elem_size)
    mesh = create_mesh(L, W, NL, NW)
    V = dolfinx.VectorFunctionSpace(mesh, ("CG", elem_order))

    bcs = create_bcs(mesh, V)
    
    # Compute Lame parameters:
    mu_val = E/(2*(1 + nu))
    mu = dolfinx.Constant(mesh, mu_val)
    lambda_ = dolfinx.Constant(mesh, lambda_)

    u = ufl.TrialFunction(V)
    v = ufl.TestFunction(V)

    def epsilon(u):
        return ufl.sym(ufl.grad(u)) # Equivalent to 0.5*(ufl.nabla_grad(u) + ufl.nabla_grad(u).T)
    def sigma(u):
        return lambda_ * ufl.nabla_div(u) * ufl.Identity(u.geometric_dimension()) + 2*mu*epsilon(u)

    f = create_load_vector(g, rho, y_rot, x_rot, mesh)
    a = ufl.inner(sigma(u), epsilon(v)) * ufl.dx
    L = ufl.dot(f, v) * ufl.dx
    
    # Delete cache of previous models:
    !rm -r /root/.cache/fenics/*
    
    problem = dolfinx.fem.LinearProblem(a, L, bcs=bcs, petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
    u = problem.solve()
    
    volumes = compute_volume(u, mesh)
    
    return (u, mesh, volumes)

## Mesh + BCs

In [4]:
def create_mesh(L, W, NL, NW):
    mesh = dolfinx.BoxMesh(MPI.COMM_WORLD,[[0.0,0.0,0.0], [L, W, W]], [NL, NW, NW], dolfinx.cpp.mesh.CellType.hexahedron)
    return mesh

In [5]:
def create_bcs(mesh, V):
    fixed = lambda x: np.isclose(x[0], 0)
    fixed_facets = dolfinx.mesh.locate_entities_boundary(mesh, mesh.topology.dim - 1, fixed)
    facet_tag = dolfinx.MeshTags(mesh, mesh.topology.dim-1, fixed_facets, 1)
    u_bc = dolfinx.Function(V)
    with u_bc.vector.localForm() as loc:
        loc.set(0)
    left_dofs = dolfinx.fem.locate_dofs_topological(V, facet_tag.dim, facet_tag.indices[facet_tag.values==1])
    bcs = [dolfinx.DirichletBC(u_bc, left_dofs)]
    return bcs

## Volume Computation

In [6]:
def compute_volume(u, mesh, quad_order=4):
    before_vol, after_vol = [], []
    ndim = mesh.geometry.x.shape[1]
    I = ufl.Identity(ndim)
    dx = ufl.Measure("dx", domain=mesh, metadata={"quadrature_degree": quad_order})
    const_funspace = dolfinx.VectorFunctionSpace(mesh, ("DG", 0), dim=1)
    const_fun = dolfinx.Function(const_funspace)
    const_fun.vector[:] = np.ones(const_fun.vector[:].shape)
    ufl.inner(const_fun,const_fun)
    before_vol.append(dolfinx.fem.assemble.assemble_scalar(ufl.inner(const_fun,const_fun)*dx))
    F = I + ufl.grad(u)
    after_vol.append(dolfinx.fem.assemble.assemble_scalar(ufl.det(F)*dx))
    return (before_vol, after_vol)

## Create Load Vector

In [7]:
# Using Euler angles - see https://www.autonomousrobotslab.com/frame-rotations-and-representations.html
# Here, y_rot = theta, x_rot = psi
def create_load_vector(g, rho, y_rot, x_rot, mesh, g_dir=(1,0,0)):
    rot_matrix = create_rot_matrix(y_rot, x_rot)
    f = rot_matrix @ (g*rho*np.array(g_dir))
    return dolfinx.Constant(mesh, f)

def create_rot_matrix(y_rot, x_rot, angle_to_rad=pi/180):
    # NB: Negative associated with y so increasing y_rot goesin 'right direction'
    theta, psi = -angle_to_rad*y_rot, angle_to_rad*x_rot
    rot_matrix = np.array([[         cos(theta),        0,          -sin(theta)],
                           [sin(psi)*sin(theta),  cos(psi), sin(psi)*cos(theta)],
                           [cos(psi)*sin(theta), -sin(psi), cos(psi)*cos(theta)]])
    return rot_matrix

## Plotting Functions

In [8]:
def plot_deformation(uh, meshio_mesh, rot_y, rot_x, title=None):
    mesh = meshio_mesh
    pyvista.start_xvfb(wait=0.05)
    topology, cell_types = dolfinx.plot.create_vtk_topology(mesh, mesh.topology.dim)
    points, u = mesh.geometry.x, uh.compute_point_values().real
    points, u = rotate_mesh(points, u, -rot_y+90, -rot_x)
    grid = pyvista.UnstructuredGrid(topology, cell_types, points)
    p = pyvista.Plotter(notebook=True, window_size=[960,480]) #
    
    title = "Deformed Configuration" if title is None else title
    p.add_text(title, name="title", position="upper_edge")
    
    grid["u"] = u
    actor_0 = p.add_mesh(grid, style="wireframe", color="k")
    warped = grid.warp_by_vector("u", factor=1.5)
    actor_1 = p.add_mesh(warped)
    
    p.show_axes()
    p.camera_position = 'xz'
    viewer = p.show(jupyter_backend='panel', return_viewer=True)
    return viewer

def rotate_mesh(points, u, y_rot, x_rot):
    rot_matrix = create_rot_matrix(y_rot, x_rot)
    rotated_points = (rot_matrix @ points.T).T
    rotated_u = (rot_matrix @ u.T).T
    return (rotated_points, rotated_u)

# Function Calls

In [47]:
W = 40 # in mm
L = 90 # in mm
E =  10 # in mPa
nu = 0.5 # dimensionless
rho = 0.00102 # in g mm^-3
g = 9.81 # in m s^-2
lambda_ = 1e4 #E*nu/((1 + nu)*(1 - 2*nu))
y_rot = 150
x_rot = 0
elem_order = 2
elem_size = W/5

import time

start = time.time()
u, mesh, volumes = perform_loading(L, W, elem_size, E, lambda_, y_rot, x_rot, nu, rho, g, elem_order)
end = time.time() - start

In [48]:
plot_deformation(u, mesh, y_rot, x_rot)

VTKRenderWindowSynchronized(vtkXOpenGLRenderWindow, color_mappers=[LinearColorMapper(id='108...], height=480, orientation_widget=True, sizing_mode='stretch_width', width=960)